### Hotel in Jeju Data
- '부킹닷컴'의 제주 내 호텔 성급, 가격, 위치, 후기를 스크래핑합니다.

In [42]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import re
import json

In [59]:
url = 'https://www.booking.com/searchresults.ko.html?aid=376440&label=bdot-SIcScZhJX6z_*YtUYg62hwS267777897793%3Apl%3Ata%3Ap1%3Ap22%2C563%2C000%3Aac%3Aap%3Aneg%3Afi%3Atikwd-325272469656%3Alp1009871%3Ali%3Adec%3Adm%3Appccp%3DUmFuZG9tSVYkc2RlIyh9YUlRwjG4dAJkHxCuUKVzpFo&sid=857a41f185409d9f6d0c6b4d32c4534e&sb=1&src=searchresults&src_elem=sb&error_url=https%3A%2F%2Fwww.booking.com%2Fsearchresults.ko.html%3Faid%3D376440%3Blabel%3Dbdot-SIcScZhJX6z_%252AYtUYg62hwS267777897793%253Apl%253Ata%253Ap1%253Ap22%252C563%252C000%253Aac%253Aap%253Aneg%253Afi%253Atikwd-325272469656%253Alp1009871%253Ali%253Adec%253Adm%253Appccp%253DUmFuZG9tSVYkc2RlIyh9YUlRwjG4dAJkHxCuUKVzpFo%3Bsid%3D857a41f185409d9f6d0c6b4d32c4534e%3Btmpl%3Dsearchresults%3Bcity%3D-714191%3Bclass_interval%3D1%3Bdest_id%3D-714191%3Bdest_type%3Dcity%3Bdr_ps%3DIDR%3Bdtdisc%3D0%3Bfrom_idr%3D1%3Bilp%3D1%3Binac%3D0%3Bindex_postcard%3D0%3Blabel_click%3Dundef%3Boffset%3D0%3Bpostcard%3D0%3Broom1%3DA%252CA%3Bsb_price_type%3Dtotal%3Bshw_aparth%3D1%3Bslp_r_match%3D0%3Bsrpvid%3De02638be83480079%3Bss_all%3D0%3Bssb%3Dempty%3Bsshis%3D0%3Btop_ufis%3D1%26%3B&ss=%EC%A0%9C%EC%A3%BC&is_ski_area=0&ssne=%EC%A0%9C%EC%A3%BC&ssne_untouched=%EC%A0%9C%EC%A3%BC&city=-714191&checkin_year=2021&checkin_month=4&checkin_monthday=24&checkout_year=2021&checkout_month=4&checkout_monthday=25&group_adults=2&group_children=0&no_rooms=1&from_sf=1'
path = '/Users/seyoung/2021/dss17/eda_etc/chromedriver'
driver = webdriver.Chrome(path)
driver.get(url)

source = driver.page_source
soup = BeautifulSoup(source, 'html.parser')

In [44]:
page_bar = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul')[0]
page_bar

<selenium.webdriver.remote.webelement.WebElement (session="0c32c7bd9d2f5bfebd93b50e89e61069", element="2c9d753a-d808-47da-aa99-6556c03fcace")>

In [45]:
# 전체 페이지 개수 확인
pages = page_bar.find_elements_by_css_selector('a > div.bui-u-inline')
last_pages = ''

for idx, page in enumerate(pages):
    print(page.text)
    if idx == len(pages)-1: # 마지막 페이지
        last_pages = page.text


1
2
3
4
5
9


In [60]:
# 현재 페이지 번호
def get_curr():
    curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
    
    return int(curr_page.text)

In [61]:
# 성급 (성급이 표시되지 않은 호텔 제외)
def get_stars(listed):
    stars = []

    for i in range(1, len(listed)):
        try:
            like = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/span/span[1]/span/span')
            a = like.get_attribute('aria-label')
            stars.append(a[0])
            
        # 성급이 표시되지 않는 호텔 
        except Exception as NoSuchElementException:
            stars.append('')
    
    return stars

In [62]:
# 가격
def get_prices(listed):
    prices = []
    
    for i in range(1, len(listed)):
        try:
            price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/div')[0]
            base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[2]/div/div/div/div/div[2]/div[1]/div[2]/div/span')[0]
            
            # strong label로 표시된 가격 포함
            if base_price.text == '':
                base_price = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[2]/div/table/tbody/tr/td[2]/div[2]/strong/label')[0]
            
            prices.append(base_price.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            prices.append('')
            
    return prices

In [63]:
# 평점
def get_grades(listed):
    grades = []
    
    for i in range(1, len(listed)):
        try:
            grade = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div[' + str(i) + ']/div[2]/div[1]/div[2]/div/div/a/div/div[1]')[0]
            grades.append(grade.text)
            
        except Exception as IndexError:
            grades.append('')
            
    return grades

In [64]:
# 호텔명
def get_names(listed):
    names = []
    
    for i in range(1, len(listed)):
        try:
            name = listed[0].find_elements_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a/span[1]')[0]
            names.append(name.text)
            
        # 중간 베너 제외
        except Exception as IndexError:
            names.append('')
            
    return names

In [55]:
# 주소
def get_addrs(listed):
    addrs = []
    
    # 새 창 열기
    for i in range(1, len(listed)):

        try:
            href = driver.find_element_by_xpath('//*[@id="hotellist_inner"]/div['+ str(i) +']/div[2]/div[1]/div[1]/div[1]/h3/a')
            href.send_keys(Keys.CONTROL + '\n') 
            
           # 새로운 탭으로 이동 (2번째 탭)
            driver.switch_to.window(driver.window_handles[1])
            addr = driver.find_elements_by_xpath('//*[@id="showMap2"]/span[1]')[0]
            addrs.append(addr.text)
            
            driver.close()
            driver.switch_to_window(driver.window_handles[0])

        except Exception as NoSuchElementException:
            addrs.append('')
            
    return addrs


['제주, 노연로 66',
 '제주, 노연로 12',
 '제주, 노연로 80',
 '',
 '제주, 도령로 83',
 '제주, 노연로 100',
 '제주, 탑동로 66',
 '제주, 중앙로 304',
 '제주, 서부두2길 20',
 '제주, 해안마을서 4길 100',
 '제주, 서부두2길 26',
 '제주, 도령로 27',
 '제주, 선돌목동길 56-26',
 '제주, 조천읍 조함해안로 502',
 '제주, 탑동 해안로 74',
 '제주, 도령로 133',
 '제주, 애월읍 곽지1길 23',
 '제주, 서사로 129',
 '제주, 조천읍 신북로 481-9',
 '제주, 중앙로 151',
 '제주, 노연로 34',
 '제주, 연삼로 14',
 '제주, 한림읍 옹포2길 10',
 '제주, 동광로 34',
 '제주, 애월읍 애월해안로 656']

In [68]:
def data_scraping():
    res_stars, res_names, res_prices, res_grades = [], [], [], []
    
    root_div = driver.find_elements_by_css_selector('#hotellist_inner')[0]
    # 한 페이지당 호출되는 목록 수
    listed = root_div.find_elements_by_xpath('//*[@id="hotellist_inner"]/div')
    
    res_stars = get_stars(listed)
    res_names = get_names(listed)
    res_prices = get_prices(listed)
    res_grades = get_grades(listed)
    
    return list(zip(res_names, res_stars, res_prices, res_grades))

data_scraping()

[('제주 오름펜션', '3', '기존 요금 ₩49,400 현재 요금 ₩44,954', '7.4'),
 ('백록담펜션', '3', '기존 요금 ₩49,500 현재 요금 ₩45,045', ''),
 ('썬랜드 호텔', '3', '₩76,977', '7.7'),
 ('거문오름 호스텔', '', '요금 ₩54,000', '7.7'),
 ('아뜨네 펜션', '3', '₩77,317', '7.4'),
 ('제주 바다성 펜션', '3', '요금 ₩35,000', ''),
 ('퓨어스테이', '', '', ''),
 ('벤티모 호텔 앤 레지던스 제주', '4', '', '9.4'),
 ('밀레니엄 호텔', '', '기존 요금 ₩31,700 현재 요금 ₩28,847', ''),
 ('대웅 황토리조트', '', '기존 요금 ₩150,091 현재 요금 ₩70,980', ''),
 ('Hannahstay in Jeju Guesthouse', '', '기존 요금 ₩60,000 현재 요금 ₩54,600', '9.1'),
 ('숨 게스트하우스 제주공항', '', '요금 ₩43,800', '8.1'),
 ('노마드인제주 게스트하우스', '', '요금 ₩50,000', '8.9'),
 ('제주도 하숙집', '', '기존 요금 ₩44,000 현재 요금 ₩40,040', '9.7'),
 ('예스준 게스트하우스', '', '요금 ₩50,000', '8.6'),
 ('그린데이 게스트하우스', '', '기존 요금 ₩38,250 현재 요금 ₩34,808', '8.3')]

In [67]:
def get_hotelDatas():
    raw_datas = []
    
    # 스크래핑 페이지 범위 (현재 페이지 ~ 마지막 페이지)
    while get_curr() <= int(last_pages):
        # scraping
        result = data_scraping()
        raw_datas.append(result)
        # print(result)

        if get_curr() == int(last_pages):
            break

        curr_page = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__pages > ul > li.bui-pagination__item.bui-pagination__item--active.sr_pagination_item.current > a > div.bui-u-inline')[0]
        print(curr_page.text,"페이지 완료")

        move_next = driver.find_elements_by_css_selector('#search_results_table > div.bui-pagination.results-paging_simplified.js-results-paging > nav > ul > li.bui-pagination__item.bui-pagination__next-arrow > a')[0]
        move_next.send_keys('\n')

        driver.implicitly_wait(1)
        time.sleep(1+random.uniform(0,1))
        
    return raw_datas

In [66]:
# 기존 요금, 현재 요금 데이터 파싱
def price_parsing(datas):
    target_prices = []
    
    for i in range(len(datas[0])):
        target_prices.append(datas[0][i][2])

    prices = [] 

    for i in range(len(target_prices)):
        re_prices = re.findall('\d+', target_prices[i])
        base_curr_prices = [] 

        if len(re_prices) == 0:
            base_curr_prices.append('')
            base_curr_prices.append('')
            
        elif len(re_prices) == 2:
            base_curr_prices.append(re_prices[0] + re_prices[1])
            base_curr_prices.append('')
            
        else:
            base_curr_prices.append(re_prices[0] + re_prices[1])
            base_curr_prices.append(re_prices[2] + re_prices[3])

        prices.append(base_curr_prices)

#     print(prices)
    return prices

price_parsing(datas)

[['49400', '44954'],
 ['49500', '45045'],
 ['76977', ''],
 ['54000', ''],
 ['77317', ''],
 ['35000', ''],
 ['', ''],
 ['', ''],
 ['31700', '28847'],
 ['150091', '70980'],
 ['60000', '54600'],
 ['43800', ''],
 ['50000', ''],
 ['44000', '40040'],
 ['50000', ''],
 ['38250', '34808']]

In [65]:
datas = get_hotelDatas()
re_prices = price_parsing(datas)

raw_dicts = {'hotel_name':'', 'star':'', 'base_price':'', 'cur_price':'', 'grade':''}
raw_datas = []
hotelName_idx, star_idx, grade_idx = 0, 1, 3

for i in range(len(datas[0])):
    
    raw_dicts['hotel_name'] = datas[0][i][hotelName_idx]
    raw_dicts['star'] = datas[0][i][star_idx]
    raw_dicts['base_price'] = re_prices[i][0]
    raw_dicts['cur_price'] = re_prices[i][1]
    raw_dicts['grade'] = datas[0][i][grade_idx]
    
    raw_datas.append(raw_dicts.copy())
    
raw_datas

[{'hotel_name': '제주 오름펜션',
  'star': '3',
  'base_price': '49400',
  'cur_price': '44954',
  'grade': '7.4'},
 {'hotel_name': '백록담펜션',
  'star': '3',
  'base_price': '49500',
  'cur_price': '45045',
  'grade': ''},
 {'hotel_name': '썬랜드 호텔',
  'star': '3',
  'base_price': '76977',
  'cur_price': '',
  'grade': '7.7'},
 {'hotel_name': '거문오름 호스텔',
  'star': '',
  'base_price': '54000',
  'cur_price': '',
  'grade': '7.7'},
 {'hotel_name': '아뜨네 펜션',
  'star': '3',
  'base_price': '77317',
  'cur_price': '',
  'grade': '7.4'},
 {'hotel_name': '제주 바다성 펜션',
  'star': '3',
  'base_price': '35000',
  'cur_price': '',
  'grade': ''},
 {'hotel_name': '퓨어스테이',
  'star': '',
  'base_price': '',
  'cur_price': '',
  'grade': ''},
 {'hotel_name': '벤티모 호텔 앤 레지던스 제주',
  'star': '4',
  'base_price': '',
  'cur_price': '',
  'grade': '9.4'},
 {'hotel_name': '밀레니엄 호텔',
  'star': '',
  'base_price': '31700',
  'cur_price': '28847',
  'grade': ''},
 {'hotel_name': '대웅 황토리조트',
  'star': '',
  'base_price': '

---

In [1]:
# 한글 설정
import matplotlib.pyplot as plt
%matplotlib inline

from matplotlib import font_manager, rc
plt.rcParams['axes.unicode_minus'] = False

f_path = "/Users/seyoung/Library/Fonts/AppleGothic.ttf"
font_name = font_manager.FontProperties(fname=f_path).get_name()
rc('font', family=font_name)